# Flask - db - angular 1

U ovom primeru bice prikazano povezivanje angular UI dela sa rest servisima implementiranim pomocu Flask servera.

```
Klijent                          Server
----------                    ------------
browser(Chrome,             python, Flask
        Firefox, IE)

angular                     [GET]/api/login  # login servis
javascript                  [GET]/api/users  # lista svih korisnika
HTML5, css                  [POST]/api/users # kreiranje korisnika
```

Struktura projekta:
```
flask-db-angular
 |
 +-run.py   // definicija rest servisa
 |
 +-static   // html, css, javascript datoteke
    + index.html  // osnovna stranica u kojoj ce se izvrsavati app
    +-app   // angular aplikacija
       +-js       // javascript kontroleri
       +-pages    // html stranice za opis UI
    +-css  // skup 'obaveznih' css datoteka
    +-js   // skup 'obaveznih' javascript datoteka
```


## run.py




In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

from flask import Flask
from flask import request
from flask import jsonify
import random
import string
import json
import mysql.connector

Novost su `random` i `string` paketi koji ce biti posebno opisani ali sad bitni.


In [ ]:
@app.route('/<path:path>')
def static_file(path):
    try:    
        return app.send_static_file(path)
    except: 
        return "Error"

@app.route('/')
def root():
    return app.send_static_file('index.html')


Definicija osnovnih ruta. Uobicajeni deo ako zelimo da FLask server 'servira' html, css i javascript datoteke.

In [ ]:
dbconfig = {
    "host": 'rpi3-djordje.local',
    "database": "Agrotrail",
    "user":     "agro",
    "password": "trail"
}

pool = mysql.connector.pooling.MySQLConnectionPool(
    pool_name = "mypool", pool_size = 3, **dbconfig)

Konektor za bazu. Preko `pool` promenljive dobijacemo konekciju prema bazi. 

Za potrebe autorizacije svakog zahteva izmedju klijentskog i serverskog dela generisacemo slucajan niz karaktera odredjene duzine i pridruzivacemo ga svakom korisniku. Taj niz koristicemo kasnije kao token u header delu svakog request-a.

U ovom primeru generisacemo token duzine 32 karaktera. 


In [2]:
def token_generator():
    N = 32
    alphabet = string.ascii_lowercase
    alphabet += string.ascii_uppercase 
    alphabet += string.digits
    return ''.join(random.choice(alphabet) for _ in range(N))

Primeri generisanih token-a:
- FkIYVXJPgCGznSr0iKFO3VM3fN5eP56J
- Bmn8jRptaPG7Z6wG7u6IkMmNy6eVvhws
- ZxK9T1lYlruwdyhpjgCJCq4cgp9nzY9j

REST servisi
### /api/login [GET]

In [ ]:
# login
@app.route("/api/login", methods=['GET'])
def api_login():
    username = request.args.get('username', None)
    password = request.args.get('password', None)
    if(username==None or password==None):
        return 'error', 404
    con = pool.get_connection()
    cur = con.cursor()
    cur.execute("""SELECT username, password 
                   FROM Users 
                   WHERE username='{0}' AND password='{1}'
                """.format(username, password))
    row = cur.fetchone()
    cur.close()
    con.close()
    if row == None:
        return 'error', 404
    else:
        token = token_generator()
        return token, 200

### /api/users [GET]

In [ ]:
# get list of all users
@app.route("/api/users", methods=['GET'])
def api_users():
    rez = []
    con = pool.get_connection()
    cur = con.cursor()
    cur.execute("SELECT * FROM Users LIMIT 100 OFFSET 0")
    rows = cur.fetchall()
    for row in rows:
        ob = {}
        for i, col in enumerate(cur.description):
            ob[col[0]] = row[i]
        rez.append(ob)
    con.close()
    return jsonify(result=rez)

### /api/users [POST]

In [ ]:
# create user
@app.route("/api/users", methods=['POST'])
def api_users_add():
    data = request.data
    usr = json.loads(data)
    con = pool.get_connection()
    cur = con.cursor(buffered=True)
    cur.execute("""
           INSERT INTO Users
             (Farms_idFarms, username, password)
           VALUES(1, '{0}', '{1}')
           """.format(usr['username'], usr['password'])
    )
    cur.execute("SELECT LAST_INSERT_ID();")
    id = cur.fetchone()
    con.commit()
    con.close()
    return jsonify(result={'id':id[0]})

Na kraju run.py datoteke poterbno je pokrenuti server:

In [ ]:
if __name__ == "__main__":
    app.run(host='0.0.0.0', port=88, debug=True)

## index.html

```html

<html>
<head>
<title>Flask-db-angular</title>
<!-- metadata -->
<meta name="viewport" content="width=device-width, initial-scale=1">
<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<meta name="keywords" content="example" />
<!-- css links -->
<link href="css/bootstrap.min.css" rel="stylesheet" type="text/css" media="all">
<link href="css/font-awesome.min.css" rel="stylesheet" type="text/css" media="all" />
<link href="css/style.css" rel="stylesheet" type="text/css" media="all">

<!-- javascript files -->
<script src="js/angular-1.5.3.js" type="text/javascript"></script>
<script src="js/ui-bootstrap-tpls-1.3.3.min.js" type="text/javascript" ></script>
<script src="js/angular-animate.min.js" type="text/javascript" ></script>
<script src="js/angular-ui-router.min.js" type="text/javascript" ></script>

<!-- app files -->
<link href="css/style-app.css" rel="stylesheet" type="text/css" media="all">

<script src="app/js/app.js"></script>
<script src="app/js/app.controller.js"></script>

</head>
<body ng-app="routerApp">

<div ui-view></div>

<!-- js files -->
<script src="js/jquery-3.1.1.min.js"></script>
<script src="js/bootstrap.min.js"></script>

</body>
</html>
```


Angualar aplikacija ucitava se pokretanjem javascript datoteke `app/js/app.js`. Angular framework koristi direktive:
- ng-app="routerApp"
- ui-view

da bi povezao angular aplikaciju i view deo za prikazivanje korisnickog interface.

###  app/js/app.js

Slicno kao u serverskom delu ovde se takodje definisu rute preko kojih se upravlja sa prikazivanjem delova korisnickog interfejsa. 

routerApp modul konfigurise se funkcijom u kojoj je opisana osnovna logika prikazivanja delova aplikacije.


```javascript
var routerApp = angular.module('routerApp', ['ui.router', 'ui.bootstrap']);

routerApp.$inject = ['AppController'];

// ------ router -------------
routerApp.config(function(\$stateProvider, \$urlRouterProvider) {
    
    \$urlRouterProvider.otherwise('/login');
    \$stateProvider
        .state('home', {
            url: '/home',
            templateUrl: 'app/pages/home.html',
            controller: 'AppController as vm'
        })
        .state('login', {
            url: '/login',
            templateUrl: 'app/pages/login.html',
            controller: 'LoginController as vm'
        })
        .state('about', {
            // we'll get to this in a bit
        });
});

```

Upravljanje elementima korisnickog interfejsa vrsi se preko kontroler funkcija. U ovom primeru koristicemo dva kontrolera:

```
- AppController     - 'app/js/app.js'
- LoginController   - 'app/js/app.controller.js'
```

routerApp se brine da poveze:
- rutu 'login'      http://localhost:8080/index.html#login
- templejt html:  app/pages/login.html
- kontroller:      LoginController as vm 


### #login

```javascript
//-------- login controller --------------
routerApp.controller('LoginController', function($scope, $http, $rootScope, $state) {
    var vm = this;
    
    vm.username = '';
    vm.password = '';
    vm.message = '';

    vm.login = function(){
        \$http.get('/api/users?username='+vm.username+'&password='+vm.password).then(
          function(resp){
            \$rootScope.loggedUser = vm.username;
            \$state.go('home');
        }, function(err){
            vm.message = err;
        });
    }

});
```
```html
<div class="container">
    <br>
	<h3 class="text-center">Login</h3>
	<div class="container">
		<div class="row">
			<div class="col-md-6 col-md-offset-3 col-sm-8 col-sm-offset-2">
				<form role="form">
					<div class="row">
						<div class="form-group col-lg-12">
							<label>Korisničko ime</label>
							<input type="text" ng-model="vm.username" 
                                   class="form-control">
						</div>
						<div class="form-group col-lg-12">
							<label>Lozinka</label>
							<input type="password" ng-model="vm.password" 
                                   class="form-control">
						</div>
					</div>
					<div class="row">
						<div class="form-group col-md-6 col-md-offset-3 
                                    col-xs-8 col-xs-offset-2 text-center">
							<button type="submit" ng-click="vm.login()" 
                                    class="btn btn-lg">Login</button>
						</div>
					</div>
					<div class="row">
						<div class="form-group col-md-6 col-md-offset-3 
                                    col-xs-8 col-xs-offset-2 text-center">
							<label>{{vm.message}}</label>
						</div>
					</div>
				</form>
			</div>
		</div>
	</div>
</div>
```